In [1]:
import MDAnalysis as md_analysis
import numpy

## MDAnalysis Universe Class
* The MDAnalysis Universe class contains all the information describing the system.
* There are no required constructor arguments, but to generating a universe will generally require a topology file containing atomic information (in this case `./cnt_atomic.data`), and optionally an MD simulation trajactory/coordinate file (in this case `./cnt_breakable_bonds_dump.lammpstrj`)
* However, some topology files can contain both atomic information **and** coordinate data (e.g. XYZ, PDB, GRO or CRD files)
* Additionally, any file you provide you need to add an additional constructor argument for the file format
* Here, we have also added the following additional constructor arguments
  * `atom_style` - Specifies the atoms ID , type and scaled x, y, z coordinates (xs, ys, zs)
  * `guess_bonds` - Instructs MDAnalysis to guess the bonds between the atoms based on their distances. This is used when bond information is not explicitly provided in the topology file, as it is here since we are using the AIREBO force field
  * `vdwradii = {'1':1.7}` - A dictionary that maps atom types to their VDW radaii. In this case, atom type 1 is assigned a VDW radius of 1.7

In [2]:
md_universe: md_analysis.Universe = md_analysis.Universe(
    "./cnt_atomic.data", 
    "./cnt_breakable_bonds_dump.lammpstrj", 
    topology_format = "data",
    format = "lammpsdump",
    atom_style = 'id type x y z',
    guess_bonds = True,
    vdwradii = {'1':1.7}
)

# MD Analysis Universe.select_atoms and AtomGroup Class
* The method `Universe.select_atoms` returns an AtomGroup object
* The AtomGroup class is essentially an ordered array of atoms, with their order reflecting their order defined in the molecular topology file
* It therefore supports looping, indexing & slicing like a standard Python list
* Additionally, the class has properties representing coordinates, velocities, masses, charges, atom names, residues etc
* It also has methods for calculating properties such as distances, angles, dihedrals, center of mass, as well as methods for atomic transformation & manipulation such as translation, rotation etc

In [3]:
cnt = md_universe.select_atoms("type 1")

## Basic MDAnalysis - Number of atoms in CNT and number of timestep frames 
print("Number of atoms =", cnt.n_atoms)
print("Number of frames =", md_universe.trajectory.n_frames)

## Basic MDAnalysis - Access indexes of atoms that are considered bonded by the MDAnalysis bond guesser
print("Bonded Atoms =", cnt.atoms.bonds.indices)

## Basic MDAnalysis - Loop over all the timestep frames of the trajectory
for timestep in md_universe.trajectory:
    print("Timestep frame ", timestep.frame)

## Basic MDAnalysis - Extract Atomic Positions
md_universe.atoms.positions

Number of atoms = 700
Number of frames = 286
Bonded Atoms = [[  0   2]
 [  0  23]
 [  0  56]
 ...
 [696 697]
 [696 699]
 [698 699]]
Timestep frame  0
Timestep frame  1
Timestep frame  2
Timestep frame  3
Timestep frame  4
Timestep frame  5
Timestep frame  6
Timestep frame  7
Timestep frame  8
Timestep frame  9
Timestep frame  10
Timestep frame  11
Timestep frame  12
Timestep frame  13
Timestep frame  14
Timestep frame  15
Timestep frame  16
Timestep frame  17
Timestep frame  18
Timestep frame  19
Timestep frame  20
Timestep frame  21
Timestep frame  22
Timestep frame  23
Timestep frame  24
Timestep frame  25
Timestep frame  26
Timestep frame  27
Timestep frame  28
Timestep frame  29
Timestep frame  30
Timestep frame  31
Timestep frame  32
Timestep frame  33
Timestep frame  34
Timestep frame  35
Timestep frame  36
Timestep frame  37
Timestep frame  38
Timestep frame  39
Timestep frame  40
Timestep frame  41
Timestep frame  42
Timestep frame  43
Timestep frame  44
Timestep frame  45
Time

array([[ 75.14728 ,  78.17872 ,  95.61408 ],
       [ 75.33008 ,  77.751114,  93.20232 ],
       [ 75.550476,  77.34152 ,  94.54224 ],
       ...,
       [ 84.66992 ,  82.24888 , 143.84988 ],
       [ 84.66992 ,  82.24888 , 147.60156 ],
       [ 84.85272 ,  81.82128 , 146.26175 ]], dtype=float32)

# Counting CNT Bonds
In order to measure the evolution of the number of bonds over time, we will loop over the simulation trajectory and manually extract the inter-atomic distance over time. 

We will have a nested loop within this simulation trajectory loop, looping over the indices of the atoms that are detected as bonded and calculate the distance between the two atoms.

In [4]:
for timestep in md_universe.trajectory:
    # `cnt.atoms.bonds.indices` is a list of tuples and the expression `index_1, index_2` unpacks each tuple
    # index_1 and index_2 represent the indices of the two atoms in each bond
    for index_1, index_2 in cnt.atoms.bonds.indices:
        
        ## `md_universe.atoms.positions` provides the positions of all atoms in the current timestep as a NumPy array
        ## `md_universe.atoms.indices == index_1` creates a boolean array where the elements are True for the atom with index `index_1` and False otherwise
        ## `md_universe.atoms.positions[md_universe.atoms.indices == index_1]` uses this boolean array to filter the positions array, returning the position of the atom with index index_1.
        position_1 = md_universe.atoms.positions[md_universe.atoms.indices == index_1][0]
        position_2 = md_universe.atoms.positions[md_universe.atoms.indices == index_2][0]

        ## Calculate bond length as Euclidean distance
        bond_length: float = numpy.sqrt(numpy.sum((position_1 - position_2)**2))

        # If bond length is larger than 1.8 Å, the bond is broken


# Extract Mean Bond Lengths & Number of Bonds
Using the same method above, we will now store the mean length of the bonds and the total number of bonds in two separate lists.

In [5]:
bond_length_vs_timestep_frame: list = []
bond_number_vs_timestep_frame: list = []

for timestep in md_universe.trajectory:
    frame = timestep.frame
    current_timestep_bond_lengths: list = []
    for index_1, index_2 in cnt.atoms.bonds.indices:
        position_1 = md_universe.atoms.positions[md_universe.atoms.indices == index_1]
        position_2 = md_universe.atoms.positions[md_universe.atoms.indices == index_2]

        bond_length; float = numpy.sqrt(numpy.sum((position_1 - position_2)**2))

        if bond_length < 1.8:
            current_timestep_bond_lengths.append(bond_length)

    mean_bond_length = numpy.mean(current_timestep_bond_lengths)
    number_of_bonds: int = len(current_timestep_bond_lengths)/2 # Divide by two to avoid counting twice

    bond_length_vs_timestep_frame.append([frame, mean_bond_length])
    bond_number_vs_timestep_frame.append([frame, number_of_bonds])

# Save 'bond length vs timestep frame' and 'bond number vs timestep frame' data to file
numpy.savetxt("bond_length_vs_timestep_frame.dat", bond_length_vs_timestep_frame)
numpy.savetxt("bond_number_vs_timestep_frame.dat", bond_number_vs_timestep_frame)